In [2]:
import pandas as pd
from transformers import pipeline

In [2]:
df = pd.read_csv('C:/Users/peni_/Desktop/proyecto/df_coments_extract.csv')

In [3]:
df.head()

,Rating,Comentario,Url,Timestamp
0,Valoración: 5 estrellas,"Todo muy bien, el lugar es nuevo moderno y lim...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.703516
1,Valoración: 5 estrellas,"Excelente alojamiento, trabajo remoto y cumpli...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.731027
2,Valoración: 5 estrellas,Un lugar estupendo,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.745032
3,Valoración: 5 estrellas,"totalmente nuevo, muy limpio personaluy atento",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.759035
4,Valoración: 5 estrellas,Me quedé en Blau mientras estaba en un viaje d...,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.773038


In [4]:
# Función para truncar los comentarios y manejar valores nulos
def truncate_coment(Comentario, max_length=512):
    if not isinstance(Comentario, str) or pd.isnull(Comentario):
        return "Reseña no disponible"
    return Comentario[:max_length]

In [5]:
# Función para convertir las estrellas en sentimientos
def convert_to_sentiment(label):
    star_rating = int(label[0])  # Extraer el número de estrellas del label
    if star_rating <= 2:
        return "Negativo"
    elif star_rating <= 3:
        return "Neutral"
    else:
        return "Positivo"


In [6]:
# Truncar comentarios
df["truncated_coment"] = df["Comentario"].apply(truncate_coment)

In [7]:
# Cargar el modelo para análisis de sentimientos
sentiment_model = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

In [8]:
# Procesar sentimientos en lotes
truncated_comments = df["truncated_coment"].tolist()  # Convertir a lista
sentiment_results = sentiment_model(truncated_comments)  # Procesar en lote

# Añadir los resultados al DataFrame
df["label"] = [res["label"] for res in sentiment_results]
df["score"] = [res["score"] for res in sentiment_results]

# Eliminamos columnas intermedias
df = df.drop(columns=["truncated_coment"])

In [9]:
df.head()

,Rating,Comentario,Url,Timestamp,label,score
0,Valoración: 5 estrellas,"Todo muy bien, el lugar es nuevo moderno y lim...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.703516,4 stars,0.805798
1,Valoración: 5 estrellas,"Excelente alojamiento, trabajo remoto y cumpli...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.731027,5 stars,0.752465
2,Valoración: 5 estrellas,Un lugar estupendo,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.745032,5 stars,0.636184
3,Valoración: 5 estrellas,"totalmente nuevo, muy limpio personaluy atento",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.759035,5 stars,0.745236
4,Valoración: 5 estrellas,Me quedé en Blau mientras estaba en un viaje d...,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.773038,4 stars,0.476711


In [10]:
# Aplicar la conversión
df["sentiment_label"] = df["label"].apply(convert_to_sentiment)
df.head()

,Rating,Comentario,Url,Timestamp,label,score,sentiment_label
0,Valoración: 5 estrellas,"Todo muy bien, el lugar es nuevo moderno y lim...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.703516,4 stars,0.805798,Positivo
1,Valoración: 5 estrellas,"Excelente alojamiento, trabajo remoto y cumpli...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.731027,5 stars,0.752465,Positivo
2,Valoración: 5 estrellas,Un lugar estupendo,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.745032,5 stars,0.636184,Positivo
3,Valoración: 5 estrellas,"totalmente nuevo, muy limpio personaluy atento",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.759035,5 stars,0.745236,Positivo
4,Valoración: 5 estrellas,Me quedé en Blau mientras estaba en un viaje d...,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.773038,4 stars,0.476711,Positivo


In [11]:
df.to_csv('df_sentiment.csv', index=False)

Dropeamos las columnas que no resultan necesarias para el usuario

In [12]:
df = df.drop(columns=['Rating', 'Comentario', 'Timestamp', 'label', 'score' ])

Cambiamos a valores númericos los sentimientos siendo positivo 1, negativo -1 y neutro 0 

despues realizamos la media de esos resultados sabiendo que si da un resultado positivo va a ser una media de sentimiento positivo y al contrario si es negativo, siendo un valor neutro si es un 0

In [13]:
# Mapear etiquetas de sentimiento a valores numéricos
sentiment_mapping = {'Positivo': 1, 'Negativo': -1, 'Neutral': 0}  # Ejemplo general
df['sentiment_numeric'] = df['sentiment_label'].map(sentiment_mapping)

# Calcular la media para cada URL única
df_mean_sentiment = df.groupby('Url')['sentiment_numeric'].mean().reset_index()

# Renombrar columnas para mayor claridad
df_mean_sentiment.rename(columns={'sentiment_numeric': 'mean_sentiment'}, inplace=True)

# Mostrar el resultado final
df_mean_sentiment.head()

,Url,mean_sentiment
0,https://www.airbnb.es/rooms/100044781045691589...,0.870968
1,https://www.airbnb.es/rooms/100051458827470719...,0.750000
2,https://www.airbnb.es/rooms/100114773709033483...,0.947368
3,https://www.airbnb.es/rooms/100133620739616372...,0.866667
4,https://www.airbnb.es/rooms/100133620739616372...,0.866667


Aqui transformamos los valores numericos de nuevo a positivo, negativo o neutro basandonos en lo que he explicado antes(valos positivo = sentimiento positivo, valor negativo = sentimiento negativo, valor 0 = sentimiento neutro)

In [14]:
# Crear una nueva columna con la etiqueta transformada de acuerdo a la media del sentimiento
df_mean_sentiment['sentiment_label_transformed'] = df_mean_sentiment['mean_sentiment'].apply(
    lambda x: 'Positivo' if x > 0 else ('Negativo' if x < 0 else 'Neutral')
)

# Mostrar el dataframe actualizado con la nueva columna
df_mean_sentiment.head()

,Url,mean_sentiment,sentiment_label_transformed
0,https://www.airbnb.es/rooms/100044781045691589...,0.870968,Positivo
1,https://www.airbnb.es/rooms/100051458827470719...,0.750000,Positivo
2,https://www.airbnb.es/rooms/100114773709033483...,0.947368,Positivo
3,https://www.airbnb.es/rooms/100133620739616372...,0.866667,Positivo
4,https://www.airbnb.es/rooms/100133620739616372...,0.866667,Positivo


In [16]:
df_mean_sentiment.head()

,Url,mean_sentiment,sentiment_label_transformed
0,https://www.airbnb.es/rooms/100044781045691589...,0.870968,Positivo
1,https://www.airbnb.es/rooms/100051458827470719...,0.750000,Positivo
2,https://www.airbnb.es/rooms/100114773709033483...,0.947368,Positivo
3,https://www.airbnb.es/rooms/100133620739616372...,0.866667,Positivo
4,https://www.airbnb.es/rooms/100133620739616372...,0.866667,Positivo


In [19]:
df_mean_sentiment = df_mean_sentiment.drop(columns=['mean_sentiment'])

unión de csv df_final_cleaned + df_sentiment_mean_Ulr

In [ ]:
df_cleaned = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_final_cleaned.csv')

In [ ]:
df_sentiment = df_mean_sentiment.rename(columns={'Url': 'urls'})

In [6]:
df_final_cleaned_semtiment = pd.merge(df_cleaned, df_sentiment, on='urls', how='inner')

In [12]:
df_final_cleaned_semtiment = df_final_cleaned_semtiment.rename(columns={'sentiment_label_transformed': 'sentiment'})

In [13]:
df_final_cleaned_semtiment.to_csv('final_cleaned_sentiment.csv', index=False)

In [14]:
df_final_cleaned_semtiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   urls                 330 non-null    object 
 1   timestamp            330 non-null    object 
 2   record_id            330 non-null    int64  
 3   titles               330 non-null    object 
 4   host_name            330 non-null    object 
 5   property_types       330 non-null    object 
 6   prices_per_night     330 non-null    float64
 7   location             247 non-null    object 
 8   ratings              330 non-null    float64
 9   num_reviews          330 non-null    float64
 10  cleaning_fee         305 non-null    float64
 11  dormitorios          328 non-null    float64
 12  camas                302 non-null    float64
 13  baños                314 non-null    float64
 14  maximum_guests       330 non-null    float64
 15  check_in_hour        300 non-null    obj